Inspired by [https://www.kaggle.com/kredy10/simple-lstm-for-text-classification](https://www.kaggle.com/kredy10/simple-lstm-for-text-classification)

Data augmentation based on: [https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/48038](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/48038)

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical, np_utils
from keras.callbacks import EarlyStopping
%matplotlib inline

import os

DATASET_PATH = './../src/datasets/MoodyLyrics4Q.csv'
LYRICS_DATASET_PATH = './ml4q_lyrics.csv'
AUGMENTED_LYRICS_DATASET_PATH = './ml4q_lyrics_augmented.csv'

# Dataset

The first thing this time will be to download lyrics and put them in a table form. Our dataset will have two columns: the lyrics and the releated emotion.

In [2]:
ml4q = pd.read_csv(DATASET_PATH)
ml4q.head()

,Index,Artist,Title,Mood
0,ML1,George Michael,I Want Your Sex,happy
1,ML2,Rob Zombie,Pussy Liquor,angry
2,ML3,Katatonia,12,sad
3,ML4,Bing Crosby,Swinging On A Star,happy
4,ML5,Ludacris,Get Back,angry


In [3]:
import lyricwikia

def download_lyric(song):
    try:
        return lyricwikia.get_lyrics(song['Artist'], song['Title'])
    except lyricwikia.LyricsNotFound:
        print('Could not download {}: {}, {}'.format(song['Index'], song['Artist'], song['Title']))
        return False

In [4]:
if os.path.exists(LYRICS_DATASET_PATH):
    ml4q_lyrics = pd.read_csv(LYRICS_DATASET_PATH)
else:
    lyrics_rows = list()

    for (i, row) in ml4q.iterrows():
        lyrics = download_lyric(row)
        if lyrics:
            lyrics_rows.append((
                row['Index'], lyrics, row['Mood']
            ))

    ml4q_lyrics = pd.DataFrame(lyrics_rows, columns=['ID', 'Lyrics', 'Emotion'])
    ml4q_lyrics.to_csv(LYRICSC_DATASET_PATH)

In [5]:
ml4q_lyrics.head()

,Unnamed: 0,ID,Lyrics,Emotion
0,0,ML1,There's things that you guess\nAnd things that...,happy
1,1,ML2,Baby:\nGimme a B\nGimme a A\nGimme a B\nGimme ...,angry
2,2,ML3,Black theatre of love\nViolet dancers cast the...,sad
3,3,ML4,Would you like to swing on a star\nCarry moonb...,happy
4,4,ML5,Hands up! Hands up\nHere's another one\nAnd a ...,angry


In [6]:
ml4q_lyrics.describe()

,Unnamed: 0
count,1948.000000
mean,973.500000
std,562.483481
min,0.000000
25%,486.750000
50%,973.500000
75%,1460.250000
max,1947.000000


In [7]:
ml4q_lyrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948 entries, 0 to 1947
Data columns (total 4 columns):
Unnamed: 0    1948 non-null int64
ID            1948 non-null object
Lyrics        1948 non-null object
Emotion       1948 non-null object
dtypes: int64(1), object(3)
memory usage: 61.0+ KB


## Data Agumentation

To improve the size of the dataset a peculiar data augmentation technique is emploied here. Specifically, while parsing each song, we add to the dataset the original lyrics and the lyrics obtained by applying the following translations:
- EN -> IT -> EN
- EN -> FR -> EN
- EN -> DE -> EN

In [10]:
from textblob import TextBlob
from textblob.translate import NotTranslated

from googletrans import Translator

trans = Translator()

def translate(lyrics, language):
    if hasattr(lyrics, "decode"):
        lyrics = lyrics.decode("utf-8")

    #text = TextBlob(lyrics)
    
    try:
        #text = text.translate(to=language)
        #text = text.translate(to="en")
        text = trans.translate(lyrics, dest=language).text
        text = trans.translate(text, dest='en').text
    except:#NotTranslated:
        return False

    return str(text)

In [11]:
languages = ['it', 'fr', 'de']

if os.path.exists(AUGMENTED_LYRICS_DATASET_PATH):
    ml4q_lyrics_agumented = pd.read_csv(AUGMENTED_LYRICS_DATASET_PATH)
else:
    lyrics_rows = list()

    for (i, row) in ml4q_lyrics.iterrows():
        lyrics = row['Lyrics']
        lyrics_rows.append((
            row['ID'], lyrics, row['Emotion']
        ))
        for l in languages:
            tr = translate(lyrics, l)
            if tr:
                lyrics_rows.append((
                    row['ID'] + l, tr, row['Emotion']
                ))

    ml4q_lyrics_augmented = pd.DataFrame(lyrics_rows, columns=['ID', 'Lyrics', 'Emotion'])
    ml4q_lyrics_augmented.to_csv(AUGMENTED_LYRICS_DATASET_PATH)

In [12]:
ml4q_lyrics_augmented.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7002 entries, 0 to 7001
Data columns (total 3 columns):
ID         7002 non-null object
Lyrics     7002 non-null object
Emotion    7002 non-null object
dtypes: object(3)
memory usage: 164.2+ KB


# Modeling

In [13]:
X = ml4q_lyrics['Lyrics']
Y = ml4q_lyrics['Emotion']

In [14]:
le = LabelEncoder()
Y = le.fit_transform(Y)
Y =  np_utils.to_categorical(Y)

In [15]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2)

## Data Preprocessing

- Tokenize the data and convert the text to sequences.
- Add padding to ensure that all the sequences have the same shape.
- There are many ways of taking the max_len and here an arbitrary length of 300 is chosen.

In [98]:
max_words = 150
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

## Model training

### RNN

In [105]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    #layer = LSTM(128, return_sequences=True)(layer)
    layer = LSTM(32)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    #layer = Dense(128, name='FC2')(layer)
    #layer = Activation('tanh')(layer)
    #layer = Dropout(0.5)(layer)
    layer = Dense(4,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [106]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 200)               0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 200, 50)           7500      
_________________________________________________________________
lstm_18 (LSTM)               (None, 32)                10624     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               8448      
_________________________________________________________________
activation_27 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 4)                 1028      
__________

In [107]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=50,
          validation_split=0.2)

Train on 1246 samples, validate on 312 samples
Epoch 1/50
1246/1246 [==============================] - 4s 3ms/step - loss: 1.3861 - acc: 0.2616 - val_loss: 1.3847 - val_acc: 0.2821
Epoch 2/50
1246/1246 [==============================] - 2s 2ms/step - loss: 1.3767 - acc: 0.3395 - val_loss: 1.3733 - val_acc: 0.3365
Epoch 3/50
1246/1246 [==============================] - 2s 2ms/step - loss: 1.3501 - acc: 0.3820 - val_loss: 1.3627 - val_acc: 0.3718
Epoch 4/50
1246/1246 [==============================] - 2s 2ms/step - loss: 1.3051 - acc: 0.4189 - val_loss: 1.3103 - val_acc: 0.4071
Epoch 5/50
1246/1246 [==============================] - 2s 2ms/step - loss: 1.2729 - acc: 0.4262 - val_loss: 1.2924 - val_acc: 0.3846
Epoch 6/50
1246/1246 [==============================] - 2s 2ms/step - loss: 1.2167 - acc: 0.4583 - val_loss: 1.3091 - val_acc: 0.3718
Epoch 7/50
1246/1246 [==============================] - 2s 2ms/step - loss: 1.1998 - acc: 0.4486 - val_loss: 1.2731 - val_acc: 0.4391
Epoch 8/50
1246

Evaluate model on test

In [108]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [109]:
accr = model.evaluate(test_sequences_matrix,Y_test)

390/390 [==============================] - 0s 959us/step


In [110]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 1.423
  Accuracy: 0.438


### Feedforward NN

In [69]:
def feedforward_nn():
    classifier = Sequential()
    classifier = Sequential()
    classifier.add(Dense(units = 256, kernel_initializer = 'random_normal', 
                         activation = 'sigmoid', input_dim = max_len))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(units = 4, kernel_initializer = 'glorot_normal', activation = 'softmax'))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
    return classifier

In [70]:
model = feedforward_nn()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               77056     
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 1028      
Total params: 78,084
Trainable params: 78,084
Non-trainable params: 0
_________________________________________________________________
